## Object Detection Model (RetinaNet) export using Nyoka

In [ ]:
from keras_retinanet.models import load_model
from PIL import ImageDraw
from nyoka import RetinanetToPmml
import requests
import warnings
warnings.filterwarnings("ignore")

### Download the pre-trained RetinaNet model

In [ ]:
model = requests.get("https://github.com/fizyr/keras-retinanet/releases/download/0.5.1/resnet50_coco_best_v2.1.0.h5")
with open('resnet50_coco_best_v2.1.0.h5','wb') as file:
    file.write(model.content)

### Load the downloaded model
The model is loaded using `load_model` function from keras_retinanet.

The model was trained with `coco` dataset and `resnet50` was used as backbone

In [ ]:
model = load_model('resnet50_coco_best_v2.1.0.h5', backbone_name='resnet50')

The pre-trained model has `score_threshold=0.05`, which means it will consider all classes whose predicted probability is greater than 5%. To remove noisy predictions, it is updated to 0.5 (50%)

In [ ]:
model.layers[-1].score_threshold = 0.5
model.save("Retinanet_with_new_threshold.h5")
print("The updated model is saved and it needs to be loaded again to reflect the change")
model = load_model("Retinanet_with_new_threshold.h5",backbone_name='resnet50')
print("The model is loaded again")

### Libraries to load and preprocess the image.

Since the model was trained using `resnet50` as backbone, we need to preprocess the image to convert it to the format used by resnet

In [ ]:
from keras.applications.resnet50 import preprocess_input
from keras.preprocessing.image import img_to_array, load_img
import numpy as np

#### Load and preprocess the image

In [ ]:
file = "test_1"
orig_img = load_img(file+'.png')
img = img_to_array(orig_img)
img = preprocess_input(img)

### Predict using the preprocessed image. The model will return boundary boxes, scores and classes/labels

In [ ]:
bboxes, scores, labels = model.predict(np.expand_dims(img, axis=0))

### Extracting valid predictions

In [ ]:
score_range=list(scores.ravel()).index(-1.0)
scores = scores.ravel()[:score_range]
labels = labels.ravel()[:score_range]
bboxes = bboxes[0][:score_range]

### List of classes used to train the model

In [ ]:
import json
classes = json.load(open("categories_coco.json",'r'))
classes = list(classes.values())

### Drawing boxes and labels on the original image

#### Draw the boxes and labels

In [ ]:
img_with_boxes=orig_img.copy()
drawer = ImageDraw.Draw(img_with_boxes)

for i in range(score_range):
    drawer.rectangle(bboxes[i],outline='red')
    drawer.text([bboxes[i][0], bboxes[i][1]],text=classes[labels[i]]+" "+"{:.2f}".format(scores[i]))

#### Original Image

In [ ]:
orig_img

#### Annotated image

In [ ]:
img_with_boxes

### Generate the PMML

The exporter needs following parameters - 
* `model` : The trained RetinaNet model
* `input_shape` : The expected shape of the image to be scored
* `input_format` : The format of input during inference
* `backbone_name` : Name of backbone used to train the model
* `trained_classes` : List of classes using which the model was trained
* `pmml_file_name` : Name of PMML file

In [ ]:
RetinanetToPmml(
    model=model,
    input_shape=(224,224,3),
    input_format='image',
    backbone_name='resnet',
    trained_classes=classes,
    pmml_file_name="RetinaNet.pmml"
)